## Set things up

In [1]:
## Import all the things

import pyglet
import pandas as pd

## Code to play sound

Popup window will open when code is executed. When you close window, sound will stop.

In [2]:
def launch_window(text, clock):
    """ Launch window with text. """
    
    window = pyglet.window.Window(caption='datasong Sound Player')
    label = pyglet.text.Label(text,
                              font_name='Whitney',
                              font_size=16,
                              x=window.width//2, y=window.height//2,
                              anchor_x='center', anchor_y='center')

    @window.event
    def on_draw():
        window.clear()
        label.draw()
        
    @window.event
    def on_close():
        clock.unschedule(update)

In [3]:
def play_background(audio_file):
    """ Play the background song. """
    
    # Instantiate player; remove any preexisting audio from queue

    global bplayer
    
    try:
        bplayer
    except NameError: bplayer = None

    if bplayer is None:
        bplayer = pyglet.media.Player()
    else:
        bplayer.delete()
    
    # Add new audio file to queue

    bsource = pyglet.media.load(audio_file, streaming=False)

    bplayer.queue(bsource) 
    bplayer.play()

In [4]:
def play_ding(audio_file):
    """ Play DING. """

    # Instantiate player; remove any preexisting audio from queue
    
    global dplayer
    
    try:
        dplayer
    except NameError: dplayer = None

    if dplayer is None:
        dplayer = pyglet.media.Player()
    else:
        dplayer.delete()
        
    # Add new audio file to queue
        
    dsource = pyglet.media.load(audio_file, streaming=False)
    dplayer.queue(dsource)
    dplayer.play()

In [5]:
def update(dt):
    """ Runs at regular intervals within event loop. """
    
    play_ding('sound-files/DING_StableB_01_1_NZD.wav')

In [6]:
class Conductor:
    """
    Class that ingests and sonifies data.
    """

    def __init__(self, data):
        self.data = data
        
        if 'DataFrame' not in str(type(self.data)): 
            raise Exception('Not a pandas DataFrame.')
    
    def background(self, background):
        """ Select genre of music to use for sonification. """
        
        self.background = background
    
    def ding(self, cols):
        """ Assign columns for DINGS. """
        
        self.ding_cols = cols
        
        # Check to make sure columns are in dataframe
        
        if type(cols) == str:
            cols = [cols]

        column_check = [col in self.data.columns for col in cols]

        if False in column_check:
            raise Exception('Column(s) to sonify not in DataFrame.')
    -
    def play(self):
        """ Sonify and play data. """
        
        # Print text to console
        
        print('Sonifying data...')
        print('Close sound player to stop playback and exit.')
        
        # Play background
        
        play_background(self.background)
        
        # Loop through DataFrame - play dings
        
        clock = pyglet.clock
        clock.schedule_interval(update, 2)
        
        # Launch window and play sound
        
        launch_window('Don\'t panic', clock)
        
        pyglet.app.run()

Run code below to test.

In [7]:
dat = pd.read_csv('data/intrusion_detection.csv')

test = Conductor(dat)
test.background('sound-files/Stable_A_16_NZD.wav')
test.play()

Sonifying data...
Close sound player to stop playback and exit.
